In [1]:
import os
import sys
import pdb
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import torch
from torch import nn
from torch.cuda.amp import GradScaler, autocast
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Adam, lr_scheduler
import torchvision
import torchvision.transforms as T

from sys import platform

DEVICE = 'mps' if platform == 'darwin' else 'cuda'
if DEVICE == 'mps':
    DOWNLOAD_PATH = '/Users/georgestoica/Downloads' 
else:
    DOWNLOAD_PATH = '/srv/share/gstoica3/checkpoints/REPAIR/'

/srv/share/gstoica3/miniconda3/envs/open-mmlab/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def save_model(model, i):
    sd = model.state_dict()
    path = os.path.join(
        DOWNLOAD_PATH,
        '%s.pth.tar' % i
    )
    torch.save(model.state_dict(), path)

def load_model(model, i):
    path = os.path.join(
        DOWNLOAD_PATH,
        '%s.pth.tar' % i
    )
    sd = torch.load(path, map_location=torch.device(DEVICE))
    model.load_state_dict(sd)

In [3]:
CIFAR_MEAN = [125.307, 122.961, 113.8575]
CIFAR_STD = [51.5865, 50.847, 51.255]
normalize = T.Normalize(np.array(CIFAR_MEAN)/255, np.array(CIFAR_STD)/255)
denormalize = T.Normalize(-np.array(CIFAR_MEAN)/np.array(CIFAR_STD), 255/np.array(CIFAR_STD))

train_transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomCrop(32, padding=4),
    T.ToTensor(),
    normalize,
])
test_transform = T.Compose([
    T.ToTensor(),
    normalize,
])
train_dset = torchvision.datasets.CIFAR10(root='/tmp', train=True,
                                        download=True, transform=train_transform)
test_dset = torchvision.datasets.CIFAR10(root='/tmp', train=False,
                                        download=True, transform=test_transform)

train_aug_loader = torch.utils.data.DataLoader(train_dset, batch_size=500, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_dset, batch_size=500, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

def _weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
#             self.shortcut = LambdaLayer(lambda x:
#                                         F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False),
                nn.BatchNorm2d(planes)
            )


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, w=1, num_classes=10, text_head=False):
        super(ResNet, self).__init__()
        self.in_planes = w*16

        self.conv1 = nn.Conv2d(3, w*16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(w*16)
        self.layer1 = self._make_layer(block, w*16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, w*32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, w*64, num_blocks[2], stride=2)
        if text_head:
            num_classes = 512
        self.linear = nn.Linear(w*64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def resnet20(w=1, text_head=False):
    return ResNet(BasicBlock, [3, 3, 3], w=w, text_head=text_head)

In [5]:
# evaluates accuracy
def evaluate(model, loader=test_loader, return_confusion=False):
    model.eval()
    correct = 0
    confusion_matrix = np.zeros((10, 10))
    with torch.no_grad(), autocast():
        for inputs, labels in loader:
            outputs = model(inputs.to(DEVICE))
            pred = outputs.argmax(dim=1)
            correct += (labels.to(DEVICE) == pred).sum().item()
            confusion_matrix[labels.cpu().numpy(), pred.cpu().numpy()] += 1
    confusion_matrix /= confusion_matrix.sum(-1, keepdims=True)
    if return_confusion:
        return correct, confusion_matrix
    else:
        return correct

# evaluates loss
def evaluate1(model, loader=test_loader):
    model.eval()
    losses = []
    with torch.no_grad(), autocast():
        for inputs, labels in loader:
            outputs = model(inputs.to(DEVICE))
            loss = F.cross_entropy(outputs, labels.to(DEVICE))
            losses.append(loss.item())
    return np.array(losses).mean()

In [6]:
# use the train loader with data augmentation as this gives better results
def reset_bn_stats(model, epochs=1, loader=train_aug_loader):
    # resetting stats to baseline first as below is necessary for stability
    for m in model.modules():
        if type(m) == nn.BatchNorm2d:
            m.momentum = None # use simple average
            m.reset_running_stats()
    # run a single train epoch with augmentations to recalc stats
    model.train()
    for _ in range(epochs):
        with torch.no_grad(), autocast():
            for images, _ in loader:
                output = model(images.to(DEVICE))

In [7]:
def do_nothing(x, mode="mean"):
    return x

def bipartite_soft_matching(
    metric: torch.Tensor,
    r: float,
    class_token: bool = False,
    distill_token: bool = False,
):
    """
    Applies ToMe with a balanced matching set (50%, 50%).
    Input size is [batch, tokens, channels].
    r indicates the ratio of tokens to remove (max 50% of tokens).
    Extra args:
     - class_token: Whether or not there's a class token.
     - distill_token: Whether or not there's also a distillation token.
    When enabled, the class token and distillation tokens won't get merged.
    """
    protected = 0
    if class_token:
        protected += 1
    if distill_token:
        protected += 1

    # We can only reduce by a maximum of 50% tokens
    t = metric.shape[1]
    r = int(r * t)
    r = min(r, (t - protected) // 2)

    if r <= 0:
        return do_nothing, do_nothing

    with torch.no_grad():
        metric = metric / metric.norm(dim=-1, keepdim=True)
        a, b = metric[..., ::2, :], metric[..., 1::2, :]
        scores = a @ b.transpose(-1, -2)
        
#         scores = torch.cov(torch.cat([a, b], dim=1)[0])[None, :r, r:t]
#         scores = -torch.cdist(a, b)

        if class_token:
            scores[..., 0, :] = -math.inf
        if distill_token:
            scores[..., :, 0] = -math.inf

        node_max, node_idx = scores.max(dim=-1)
        edge_idx = node_max.argsort(dim=-1, descending=True)[..., None]

        unm_idx = edge_idx[..., r:, :]  # Unmerged Tokens
        src_idx = edge_idx[..., :r, :]  # Merged Tokens
        dst_idx = node_idx[..., None].gather(dim=-2, index=src_idx)

        if class_token:
            # Sort to ensure the class token is at the start
            unm_idx = unm_idx.sort(dim=1)[0]

    def merge(x: torch.Tensor, mode="mean") -> torch.Tensor:
        src, dst = x[..., ::2, :], x[..., 1::2, :]
        n, t1, c = src.shape
        unm = src.gather(dim=-2, index=unm_idx.expand(n, t1 - r, c))
        src = src.gather(dim=-2, index=src_idx.expand(n, r, c))
        dst = dst.scatter_reduce(-2, dst_idx.expand(n, r, c), src, reduce=mode)

        if distill_token:
            return torch.cat([unm[:, :1], dst[:, :1], unm[:, 1:], dst[:, 1:]], dim=1)
        else:
            return torch.cat([unm, dst], dim=1)

    def unmerge(x: torch.Tensor) -> torch.Tensor:
        unm_len = unm_idx.shape[1]
        unm, dst = x[..., :unm_len, :], x[..., unm_len:, :]
        n, _, c = unm.shape

        src = dst.gather(dim=-2, index=dst_idx.expand(n, r, c))

        out = torch.zeros(n, metric.shape[1], c, device=x.device, dtype=x.dtype)

        out[..., 1::2, :] = dst
        out.scatter_(dim=-2, index=(2 * unm_idx).expand(n, unm_len, c), src=unm)
        out.scatter_(dim=-2, index=(2 * src_idx).expand(n, r, c), src=src)

        return out

    return merge, unmerge

def permutation_matching(metric, r):
    with torch.no_grad():
        metric = metric / metric.norm(dim=-1, keepdim=True)
        a, b = metric[..., ::2, :], metric[..., 1::2, :]
        scores = -(a @ b.transpose(-1, -2))
#     pdb.set_trace()
    row_ind, col_ind = scipy.optimize.linear_sum_assignment(scores[0].cpu().numpy().T)
    row_ind = torch.from_numpy(col_ind)[None, :, None].to(metric.device)
    
#     print(row_ind, col_ind)
    
    def merge(x: torch.Tensor, mode="mean") -> torch.Tensor:
        src, dst = x[..., ::2, :], x[..., 1::2, :]
        n, t1, c = src.shape
        
        src = src.gather(dim=-2, index=row_ind.expand(n, t1, c))
        
        if mode == "sum":
            return dst * 0. + src
        elif mode == "mean":
            return (dst * 0. + src) #/ 2
        else:
            return 1 / 0
    
    def unmerge(x):
        pdb.set_trace()
        n, r, c = x.shape
        out = torch.zeros(n, metric.shape[1], c, device=x.device, dtype=x.dtype)

        out[..., 1::2, :] = x
        out.scatter_(dim=-2, index=(2 * row_ind).expand(n, r, c), src=x)

        return out
    
    return merge, unmerge

# bipartite_soft_matching = permutation_matching

def svd_matching(metric, r):
    U, S, V = torch.svd(metric[0])
    return U[None, :, :r], (S @ V.T)[None, :r, :]
        

In [8]:
def strip_param_suffix(name):
    return name.replace('.weight', '').replace('.bias', '')

def naively_compress_model(model, ratio):
    new_sd = {}
    for key, val in model.state_dict().items():
        shape = val.shape
        if len(shape) == 4:
            val = val.flatten(2).permute(2, 0, 1)
            merge, unmerge = bipartite_soft_matching(val, r=ratio)
            val = unmerge(merge(val))
            if not key.startswith('conv1'):
                val = val.transpose(2, 1)
                merge, unmerge = bipartite_soft_matching(val, r=ratio)
                val = unmerge(merge(val)).transpose(2, 1)
            val = val.permute(1, 2, 0).reshape(*shape)
        elif len(shape) == 2:
            val = val[None]
            merge, unmerge = bipartite_soft_matching(val, r=0)
            val = unmerge(merge(val)).transpose(2, 1)
            merge, unmerge = bipartite_soft_matching(val, r=ratio)
            val = unmerge(merge(val)).transpose(2, 1)[0]
        elif len(shape) == 1:
            val = val[None, :, None]
            merge, unmerge = bipartite_soft_matching(val, r=ratio)
            val = unmerge(merge(val))[0, :, 0]
        new_sd[key] = val
    return new_sd

In [9]:
modela = resnet20(w=4).to(DEVICE)
modelb = resnet20(w=4).to(DEVICE)
load_model(modela, 'resnet20x4_v1')
load_model(modelb, 'resnet20x4_v2')

evaluate(modela), evaluate(modelb)

(9536, 9510)

In [10]:
def make_mats(merge, unmerge, n, t, r):
    
    merge_mat   = merge(torch.eye(t, device=DEVICE)[None, ...].expand(n, t, t))
    unmerge_mat = unmerge(torch.eye(t-r, device=DEVICE)[None, ...].expand(n, t-r, t-r))
    return merge_mat, unmerge_mat

def prep_conv(weight):
    return weight.flatten(1)[None, ...]

def unprep_conv(weight, k=3):
    weight = weight[0]
    o, *_ = weight.shape
    return weight.reshape(o, -1, k, k)

def make_mats(merge, unmerge, n, t, r):
    merge_mat   = merge(torch.eye(t, device=DEVICE)[None, ...].expand(n, t, t))
    unmerge_mat = unmerge(torch.eye(t-r, device=DEVICE)[None, ...].expand(n, t-r, t-r))
    return merge_mat, unmerge_mat

state_dict = {}

def interleave_vals(tensor1, tensor2, dim=1):
    # Assume tensor is of shape [B,H,D]
    return torch.cat((tensor1.unsqueeze(dim+1), tensor2.unsqueeze(dim+1)), dim=dim+1).flatten(dim, dim+1)
#     return torch.cat((tensor1, tensor2), dim=dim)

def unterleave_vals(tensor):
    return tensor[:, ::2, :], tensor[:, 1::2, :]
#     return tensor.chunk(2, dim=1)


def merge_inconv(state_dict, prefix, a_conv, b_conv):
    a_c1 = prep_conv(a_conv.weight)
    b_c1 = prep_conv(b_conv.weight)
    
    c_c1 = interleave_vals(a_c1, b_c1)
    merge, unmerge = permutation_matching(c_c1, r=0.5)
    
    _, t, _ = c_c1.shape
    r = int(0.5*t)
    
    _, out_unmerge = make_mats(merge, unmerge, 1, t, r)
    
    c_c1 = unprep_conv(merge(c_c1))
    
    state_dict[prefix + ".weight"] = c_c1
    
    return merge, out_unmerge

def merge_conv(state_dict, prefix, a_conv, b_conv, unmerge, out_merge=None, eps=1e-7):
    def move_kernel_to_output(x): # [out, in, h, w]
        return x.transpose(0, 1).flatten(1)[None, ...] # output: [1, in, w*h*out]
    
    out, _in, k, _ = a_conv.weight.shape
    
    a_c1 = move_kernel_to_output(a_conv.weight)
    b_c1 = move_kernel_to_output(b_conv.weight)
    
    unmerge_a, unmerge_b = unterleave_vals(unmerge)
#     pdb.set_trace()
    a_c1 = unmerge_a.transpose(-1, -2) @ a_c1
    b_c1 = unmerge_b.transpose(-1, -2) @ b_c1
    
    def move_kernel_to_input(x):
        return x.transpose(1, 2).reshape(1, -1, k*k*x.shape[1]) # [1, out, h*w*in]
    
    c_c1 = interleave_vals(move_kernel_to_input(a_c1), move_kernel_to_input(b_c1))
    
    if out_merge is None:
        out_merge, out_unmerge = permutation_matching(c_c1, r=0.5)
    else:
        out_unmerge = None
    
    _, t, _ = c_c1.shape
    r = int(0.5*t)
    
    if out_unmerge is not None:
        _, out_unmerge = make_mats(out_merge, out_unmerge, 1, t, r)
    
    c_c1 = out_merge(c_c1)
    c_c1 = c_c1.reshape(1, out, k, k, _in)[0].permute(0, 3, 1, 2)
    
    state_dict[prefix + ".weight"] = c_c1
    return out_merge, out_unmerge


def merge_bn(state_dict, prefix, a_bn, b_bn, merge):
    # weight, bias, running_mean, running_var
    
    a_stats = torch.stack([a_bn.weight, a_bn.bias, a_bn.running_mean], dim=1)[None, ...]
    b_stats = torch.stack([b_bn.weight, b_bn.bias, b_bn.running_mean], dim=1)[None, ...]
    
    c_stats = interleave_vals(a_stats, b_stats)
    c_stats = merge(c_stats)[0]
    
    c_weight, c_bias, c_mean = c_stats.unbind(dim=-1)
    
    c_var = interleave_vals(a_bn.running_var[None, :, None], b_bn.running_var[None, :, None])
    
    ones = c_var * 0 + 1
    c_denom = merge(ones, mode="sum")
    c_var = merge(c_var, mode="sum")
    c_var = c_var / (c_denom ** 2)
    c_var = c_var[0, :, 0]
    
    state_dict[prefix + ".weight"] = c_weight
    state_dict[prefix + ".bias"] = c_bias
    state_dict[prefix + ".running_mean"] = c_mean
    state_dict[prefix + ".running_var"] = c_var

def merge_block(state_dict, prefix, a, b, conv1_merge, conv1_unmerge):
    c1_merge, c1_unmerge = merge_conv(state_dict, prefix + ".conv1", a.conv1, b.conv1, conv1_unmerge)
    merge_bn(state_dict, prefix + ".bn1", a.bn1, b.bn1, c1_merge)
    c2_merge, c2_unmerge = merge_conv(state_dict, prefix + ".conv2", a.conv2, b.conv2, c1_unmerge, out_merge=conv1_merge)
    merge_bn(state_dict, prefix + ".bn2", a.bn2, b.bn2, c2_merge)
    
def merge_block_shortcut(state_dict, prefix, a, b, conv1_merge, conv1_unmerge):
    c1_merge, c1_unmerge = merge_conv(state_dict, prefix + ".conv1", a.conv1, b.conv1, conv1_unmerge)
    merge_bn(state_dict, prefix + ".bn1", a.bn1, b.bn1, c1_merge)
    c2_merge, c2_unmerge = merge_conv(state_dict, prefix + ".conv2", a.conv2, b.conv2, c1_unmerge)
    merge_bn(state_dict, prefix + ".bn2", a.bn2, b.bn2, c2_merge)
    
    s_merge, s_unmerge = merge_conv(state_dict, prefix + ".shortcut.0", a.shortcut[0], b.shortcut[0], conv1_unmerge, out_merge=c2_merge)
    merge_bn(state_dict, prefix + ".shortcut.1", a.shortcut[1], b.shortcut[1], s_merge)
    return s_merge, c2_unmerge

class conv_wrapper:
    def __init__(self, linear):
        self.weight = linear.weight[:, :, None, None]

def merge_resnet20(state_dict, a, b): #, merge_output=False):
    conv1_merge, conv1_unmerge = merge_inconv(state_dict, "conv1", a.conv1, b.conv1)
    merge_bn(state_dict, "bn1", a.bn1, b.bn1, conv1_merge)
    
    for i in range(3):
        merge_block(state_dict, f"layer1.{i}", a.layer1[i], b.layer1[i], conv1_merge, conv1_unmerge)
    
    conv1_merge, conv1_unmerge = merge_block_shortcut(state_dict, "layer2.0", a.layer2[0], b.layer2[0], conv1_merge, conv1_unmerge)
    for i in range(1, 3):
        merge_block(state_dict, f"layer2.{i}", a.layer2[i], b.layer2[i], conv1_merge, conv1_unmerge)
        
    conv1_merge, conv1_unmerge = merge_block_shortcut(state_dict, "layer3.0", a.layer3[0], b.layer3[0], conv1_merge, conv1_unmerge)
    for i in range(1, 3):
        merge_block(state_dict, f"layer3.{i}", a.layer3[i], b.layer3[i], conv1_merge, conv1_unmerge)

#     if merge_output:
    merge_conv(state_dict, "linear", conv_wrapper(a.linear), conv_wrapper(b.linear), conv1_unmerge)
    state_dict["linear.weight"] = state_dict["linear.weight"][:, :, 0, 0]
#     else:
#         c_linear = interleave_vals(a.linear.weight.mT[None, ...], b.linear.weight.mT[None, ...])
#         c_linear = conv1_merge(c_linear)[0].mT
#         state_dict["linear.weight"] = c_linear
    state_dict["linear.bias"] = (a.linear.bias + b.linear.bias) / 2

# in_merge, _ = bipartite_soft_matching(torch.rand(1, 128, 20, device=DEVICE), r=0.5)

In [ ]:
def permutation_matching(metric, r):
    with torch.no_grad():
        metric = metric / metric.norm(dim=-1, keepdim=True)
        a, b = metric[..., ::2, :], metric[..., 1::2, :]
        scores = -(a @ b.transpose(-1, -2))
#     pdb.set_trace()
    row_ind, col_ind = scipy.optimize.linear_sum_assignment(scores[0].cpu().numpy().T)
    row_ind = torch.from_numpy(col_ind)[None, :, None].to(metric.device)
    
#     print(row_ind, col_ind)
    
    def merge(x: torch.Tensor, mode="mean") -> torch.Tensor:
        src, dst = x[..., ::2, :], x[..., 1::2, :]
        n, t1, c = src.shape
        
        src = src.gather(dim=-2, index=row_ind.expand(n, t1, c))
        
        if mode == "sum":
            return dst * 0. + src
        elif mode == "mean":
            return (dst * 0. + src) #/ 2
        else:
            return 1 / 0
    
    def unmerge(x):
        n, r, c = x.shape
        out = torch.zeros(n, metric.shape[1], c, device=x.device, dtype=x.dtype)

        out[..., 1::2, :] = x #torch.zeros_like(x, device=DEVICE)
        out.scatter_(dim=-2, index=(2 * row_ind).expand(n, r, c), src=x)

        return out
    
    return merge, unmerge

def merge_conv(state_dict, prefix, a_conv, b_conv, unmerge, out_merge=None, eps=1e-7):
    def move_kernel_to_output(x): # [out, in, h, w]
        return x.transpose(0, 1).flatten(1)[None, ...] # output: [1, in, w*h*out]
    
    out, _in, k, _ = a_conv.weight.shape
    
    a_c1 = move_kernel_to_output(a_conv.weight)
    b_c1 = move_kernel_to_output(b_conv.weight)
    
    unmerge_a, unmerge_b = unterleave_vals(unmerge)
    a_c1 = unmerge_a @ a_c1
    b_c1 = unmerge_b @ b_c1
    
    def move_kernel_to_input(x):
        return x.transpose(1, 2).reshape(1, -1, k*k*x.shape[1]) # [1, out, h*w*in]
    
    c_c1 = interleave_vals(move_kernel_to_input(a_c1), move_kernel_to_input(b_c1))
    
    if out_merge is None:
        out_merge, out_unmerge = permutation_matching(c_c1, r=0.5)
    else:
        out_unmerge = None
    
    _, t, _ = c_c1.shape
    r = int(0.5*t)
    
    if out_unmerge is not None:
        _, out_unmerge = make_mats(out_merge, out_unmerge, 1, t, r)
    
    c_c1 = out_merge(c_c1)
    c_c1 = c_c1.reshape(1, out, k, k, _in)[0].permute(0, 3, 1, 2)
    
    state_dict[prefix + ".weight"] = c_c1
    return out_merge, out_unmerge

In [ ]:
is_zero = lambda x: torch.allclose(x, torch.zeros((x.shape[1], x.shape[2]), device=DEVICE))
is_I = lambda x: torch.allclose(x, torch.eye(x.shape[1], device=DEVICE))

In [ ]:
state_dict = {}
modelc = resnet20(w=4).to(DEVICE)
# merge_conv(state_dict, "layer1.0.conv1", modela.layer1[0].conv1, modelb.layer1[0].conv1, in_merge)
merge_resnet20(state_dict, modelb, modela)
modelc.load_state_dict(state_dict)
# reset_bn_stats(modelc)
evaluate(modelc)